In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
BRFRSS_2015 = "2015.csv"


# Read the School Data and Student Data and store into a Pandas DataFrame
df_15 = pd.read_csv(BRFRSS_2015)
df_15.head(30)

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,b'01202015',b'01',b'20',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,b'02012015',b'02',b'01',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
5,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,2.0
6,1.0,1.0,b'01052015',b'01',b'05',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,1.0,1.0,2.0,1.0,1.0
7,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,b'01132015',b'01',b'13',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0
9,1.0,1.0,b'01302015',b'01',b'30',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0


In [3]:
df_15.shape

(441456, 330)

In [5]:
df_15.columns.to_list()

['_STATE',
 'FMONTH',
 'IDATE',
 'IMONTH',
 'IDAY',
 'IYEAR',
 'DISPCODE',
 'SEQNO',
 '_PSU',
 'CTELENUM',
 'PVTRESD1',
 'COLGHOUS',
 'STATERES',
 'CELLFON3',
 'LADULT',
 'NUMADULT',
 'NUMMEN',
 'NUMWOMEN',
 'CTELNUM1',
 'CELLFON2',
 'CADULT',
 'PVTRESD2',
 'CCLGHOUS',
 'CSTATE',
 'LANDLINE',
 'HHADULT',
 'GENHLTH',
 'PHYSHLTH',
 'MENTHLTH',
 'POORHLTH',
 'HLTHPLN1',
 'PERSDOC2',
 'MEDCOST',
 'CHECKUP1',
 'BPHIGH4',
 'BPMEDS',
 'BLOODCHO',
 'CHOLCHK',
 'TOLDHI2',
 'CVDINFR4',
 'CVDCRHD4',
 'CVDSTRK3',
 'ASTHMA3',
 'ASTHNOW',
 'CHCSCNCR',
 'CHCOCNCR',
 'CHCCOPD1',
 'HAVARTH3',
 'ADDEPEV2',
 'CHCKIDNY',
 'DIABETE3',
 'DIABAGE2',
 'SEX',
 'MARITAL',
 'EDUCA',
 'RENTHOM1',
 'NUMHHOL2',
 'NUMPHON2',
 'CPDEMO1',
 'VETERAN3',
 'EMPLOY1',
 'CHILDREN',
 'INCOME2',
 'INTERNET',
 'WEIGHT2',
 'HEIGHT3',
 'PREGNANT',
 'QLACTLM2',
 'USEEQUIP',
 'BLIND',
 'DECIDE',
 'DIFFWALK',
 'DIFFDRES',
 'DIFFALON',
 'SMOKE100',
 'SMOKDAY2',
 'STOPSMK2',
 'LASTSMK2',
 'USENOW3',
 'ALCDAY5',
 'AVEDRNK2',
 'DRNK3GE

In [6]:
df_15.TOLDHI2.value_counts()

2.0    218771
1.0    159970
7.0      3037
9.0       524
Name: TOLDHI2, dtype: int64

In [7]:
df_15.BLOODCHO.value_counts()

1.0    382302
2.0     49673
7.0      9279
9.0       202
Name: BLOODCHO, dtype: int64

In [8]:
df_15._SMOKER3.value_counts()

4.0    239608
3.0    122277
1.0     43583
2.0     17998
9.0     17990
Name: _SMOKER3, dtype: int64

In [9]:
df_15.CVDSTRK3.value_counts()

2.0    421897
1.0     18269
7.0      1146
9.0       144
Name: CVDSTRK3, dtype: int64

In [10]:
df_15._STATE.value_counts()

20.0    23236
31.0    17561
27.0    16761
53.0    16116
48.0    14697
8.0     13537
6.0     12601
24.0    12598
36.0    12357
39.0    11929
9.0     11899
45.0    11607
34.0    11465
49.0    11401
12.0     9739
25.0     9294
23.0     9063
26.0     8935
21.0     8806
51.0     8646
1.0      7950
4.0      7946
29.0     7307
46.0     7221
15.0     7163
33.0     7022
40.0     6943
35.0     6734
37.0     6698
50.0     6489
19.0     6227
44.0     6206
55.0     6188
18.0     6067
30.0     6051
28.0     6035
47.0     5979
54.0     5957
16.0     5802
42.0     5740
56.0     5492
72.0     5405
41.0     5359
17.0     5289
5.0      5256
38.0     4972
22.0     4716
13.0     4678
10.0     4070
11.0     3994
2.0      3657
32.0     2926
66.0     1669
Name: _STATE, dtype: int64